## 상권 데이터_대분류 수집

In [3]:
import requests
import pandas as pd

In [4]:
Service_key = 'LVR82OMbkY0s4ySfvPsAAzVklADnEDLtwD4dgRH/T434bdVKyQhzWZew+9q669ncqTMgLzDna1iE7u4OTUKPGg=='

In [5]:
# 상권 데이터를 수집하기 위해 해당 url 할당
# 파라미터 설정(상가 상권데이터의 참고문서의 파라미터 구성 확인)
url = 'http://apis.data.go.kr/B553077/api/open/sdsc2/storeListInUpjong'
payload = dict(divId='indsLclsCd', key='I2', numOfRows=1000, pageNo=1, type='json',servicekey=Service_key)

# requests 모듈 get 방식으로 받아서 받아온 데이터 r에 할당
r = requests.get(url, params=payload)

# r에 할당된 전체 url과 상태 코드 확인
# json 형식의 데이터를 딕셔너리로 변환하여 할당
print(r.url)
print(r.status_code)
data = r.json()

# 전체 데이터의 수 확인 및 n_data에 할당
n_data = data['body']['totalCount']
# print(n_data)

# 전체 페이지 수 계산 및 total_pages에 할당
total_pages = (int(data['body']['totalCount'])//1000)+1
print(total_pages)


http://apis.data.go.kr/B553077/api/open/sdsc2/storeListInUpjong?divId=indsLclsCd&key=I2&numOfRows=1000&pageNo=1&type=json&servicekey=LVR82OMbkY0s4ySfvPsAAzVklADnEDLtwD4dgRH%2FT434bdVKyQhzWZew%2B9q669ncqTMgLzDna1iE7u4OTUKPGg%3D%3D
200
870


In [20]:
# header의 columns와 zip해서 나타내기
data.keys()
# data['body']['items']
# data['header']['columns']

dict_keys(['header', 'body'])

In [6]:
# 받아온 데이터를 딕셔너리 형태로 저장하여 데이터프레임으로 변환
# 데이터프레임의 컬럼명 변환
def json_unpack(data):
    # 받아온 데이터 딕셔너리 형태로 저장하여 result에 할당
    result = {}
    # 변경할 데이터프레임 컬럼 할당
    header_columns = data[0]['header']['columns']
    
    # 데이터 변환
    for page_data in data:
        for dicts in page_data['body']['items']:
        #     print(dicts)
            for key, value in dicts.items():
        #         print(key, value)
                result.setdefault(key, []).append(value)
        # print(result)
    df = pd.DataFrame(result)
    
    # 컬럼명 변경 가능
    df.columns = header_columns
    
    return df

In [9]:
print(n_data)
print(total_pages)

869601
870


In [42]:
import time

# 전체 페이지의 데이터 수집
def public_small_business(category):
    result_json = []
    page = 1
    
    while True:
        
        # 상권 데이터를 수집하기 위해 해당 url 할당
        # 파라미터 설정(상가 상권데이터의 참고문서의 파라미터 구성 확인)
        url = 'http://apis.data.go.kr/B553077/api/open/sdsc2/storeListInUpjong'
        
        # key는 대분류별로 받아서 확인하기 위함
        payload = dict(divId='indsLclsCd', key=f"{category}", numOfRows=1000, pageNo=page, type='json',servicekey=Service_key)
        
        # requests 모듈 get 방식으로 받아서 받아온 데이터 r에 할당
        r = requests.get(url, params=payload)
        
        
        # r에 할당된 전체 url과 상태 코드 확인
        # json 형식의 데이터를 딕셔너리로 변환하여 할당
        print(r.url, end='\r')
        print(r.status_code, end='\r')
        data = r.json()

        # 전체 데이터의 수 확인 및 n_data에 할당
        # 전체 페이지 수 계산 및 total_pages에 할당
        n_data = data['body']['totalCount']
        total_pages = (int(data['body']['totalCount'])//1000)+1
        
        # 받아온 데이터를 리스트 형식으로 result_json에 할당
        result_json.append(data)
#         print(f"{page} / {total_pages} 수집 중")
        time.sleep(0.2)
        
        # page가 전체 페이지보다 작은 경우 +1씩 더함
        # 위 경우가 아닌 경우 멈춤
        if page < total_pages:
            page += 1
        else:
            break

    return result_json


In [43]:
major = ['G2','I1','I2','L1','M1','L1','P1','Q1','R1','S2']
all_data = []

for category in major:
#     print(category)
    result_by_category = public_small_business(category)
    df_by_category = json_unpack(result_by_category)
    all_data.append(df_by_category)
#     print(f"{len(all_data)} / {len(major)} 수집", end='\r')
    
all_df = pd.concat(all_data, ignore_index=True)
# display(all_df)

## 상권 데이터 수집

In [39]:
def unpack_data(response):
    result = {}
    for item in response['body']['items']:
        for key, value in zip(response['header']['columns'], item.values()):
            result.setdefault(key, []).append(value)
    df = pd.DataFrame(result)
    return df

In [40]:
page = 1
result_dfs = []

while True:
    url = "http://apis.data.go.kr/B553077/api/open/sdsc2/storeListInUpjong"
    service_key = "LVR82OMbkY0s4ySfvPsAAzVklADnEDLtwD4dgRH/T434bdVKyQhzWZew+9q669ncqTMgLzDna1iE7u4OTUKPGg=="
    payload = dict(servicekey=service_key, divId="indsSclsCd", 
                  key="I21006", numOfRows=1000, pageNo=page, type="json")
    r = requests.get(url, params=payload)
    print(r.url, end="\r")
    print(r.status_code, end="\r")
    
    response = r.json()
    total_page = (response['body']['totalCount'] // 1000) + 1
    result_dfs.append(unpack_data(response))
    print(f"{page}/{total_page} 수집중", end="\r")
    if page < total_page:
        page += 1
    else:
        break
    time.sleep(0.2)
        
result = pd.concat(result_dfs)    
result = result.reset_index(drop=True)
result
# result.to_csv("공공데이터_상권정보_202504.csv", encoding="utf-8", index=False)

,상가업소번호,상호명,지점명,상권업종대분류코드,상권업종대분류명,상권업종중분류코드,상권업종중분류명,상권업종소분류코드,상권업종소분류명,표준산업분류코드,...,건물관리번호,건물명,도로명주소,구우편번호,신우편번호,동정보,층정보,호정보,경도,위도
0,MA010120220800445252,가마로닭강정,,I2,음식,I210,기타 간이,I21006,치킨,I56193,...,1159010500102240001017670,제,서울특별시 동작구 서달로 151,156861,06973,,1,,126.961535,37.506975
1,MA010120220800445378,수원왕갈비통닭,보라매점,I2,음식,I210,기타 간이,I21006,치킨,I56213,...,1159010800104150005004103,,서울특별시 동작구 여의대방로24가길 12,156811,07055,,1,,126.921008,37.498774
2,MA010120220800430361,페리카나,,I2,음식,I210,기타 간이,I21006,치킨,I56193,...,1159010700102130007006364,,서울특별시 동작구 사당로 168,156879,07030,,,,126.967572,37.486813
3,MA010120220800445715,치킨89,,I2,음식,I210,기타 간이,I21006,치킨,I56193,...,1159010900103460008001825,노블레스빌,서울특별시 동작구 보라매로 98-5,156847,07053,,,,126.928484,37.497932
4,MA010120220800439275,또바기치킨,,I2,음식,I210,기타 간이,I21006,치킨,I56193,...,1156013200103470294013808,,서울특별시 영등포구 가마산로 439,150848,07382,,,,126.905292,37.505322
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42090,MA0106202410A0530080,다사랑치킨,에코시티점,I2,음식,I210,기타 간이,I21006,치킨,I56111,...,4511312200113300014000001,,전북특별자치도 전주시 덕진구 세병2길 32-18,561302,55149,,1,,127.125791,35.872366
42091,MA0106202410A0842347,푸라닭이천,마장점,I2,음식,I210,기타 간이,I21006,치킨,I56111,...,4150034021102830015036863,,경기도 이천시 마장면 오천로 33,467814,17387,,,,127.352308,37.253954
42092,MA0106202410A0741260,보드람치킨,쌍용점,I2,음식,I210,기타 간이,I21006,치킨,I56111,...,4413310700108230000055345,,충청남도 천안시 서북구 쌍용대로 119,331949,31149,,1,,127.132527,36.808137
42093,MA0106202410A2090834,찐이야숯불치킨,태전점,I2,음식,I210,기타 간이,I21006,치킨,I56111,...,2723012600102540030045203,,대구광역시 북구 태전로 53,702863,41456,,1,,128.543405,35.925012


In [37]:
# !pip install openpyxl
# conda install openpyxl

In [38]:
ori_data = pd.read_excel('./data/소상공인시장진흥공단_상가(상권)정보_업종분류(2302)_및_연계표_v1.0.xlsx', engine='openpyxl', header=[1])
ori_data

,대분류코드,대분류명,중분류코드,중분류명,소분류코드,소분류명
0,G2,소매업,G202,자동차 부품 및 내장품 소매업,G20201,타이어 소매업
1,G2,소매업,G202,자동차 부품 및 내장품 소매업,G20202,자동차 부품 소매업
2,G2,소매업,G203,모터사이클 및 부품 소매업,G20301,모터사이클 및 부품 소매업
3,G2,소매업,G204,종합 소매업,G20404,슈퍼마켓
4,G2,소매업,G204,종합 소매업,G20405,편의점
...,...,...,...,...,...,...
242,S2,수리 및 개인 서비스업,S209,세탁업,S20902,셀프 빨래방
243,S2,수리 및 개인 서비스업,S210,장례식장 및 관련 서비스업,S21001,장례식장
244,S2,수리 및 개인 서비스업,S210,장례식장 및 관련 서비스업,S21002,화장터/묘지/납골당
245,S2,수리 및 개인 서비스업,S211,기타 개인서비스,S21101,예식장업
